# EDA for Big Data Derby Competition

## MAIN

### Creating paths to diferent folders

In [1]:
import os
from pathlib import Path

# Get current work directory of file (Notebook dir)
NOTEBOOK_DIR=Path(os.path.abspath(os.getcwd()))
# Root path of repo
WORK_DIR=NOTEBOOK_DIR.parent.parent
# Path to Datasets
DATA_DIR=WORK_DIR / 'data'

# Noob printing to check if I did this shit right
print("Notebook dir: ", NOTEBOOK_DIR)
print("Repo dir: ", WORK_DIR)
print("Data dir: ", DATA_DIR)

Notebook dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle\notebooks\nad
Repo dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle
Data dir:  c:\Users\Nad\Documents\Programación\Kaggle\big-data-derby-kaggle\data


### Loading csv into dataframes

In [2]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

# horse_data = pd.read_csv(DATA_DIR / 'horse_ids.csv', index_col=0)
df_tracking = pd.read_csv(DATA_DIR / 'ordered_tracking.csv')
df_tracking = df_tracking.groupby(['race_date','race_number','program_number','trakus_index']).min()


#### Loader functions

In [3]:
def load_data():
    complete_data_headers = ['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id',
    'course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','position_at_finish']
    dtypes = {'race_date': 'string',
            'program_number': 'string',
            }
    complete_data= pd.read_csv(DATA_DIR / 'nyra_2019_complete.csv', names=complete_data_headers, dtype=dtypes)

    loaded = complete_data.drop(['trakus_index','latitude','longitude','purse','post_time'], axis = 1)
    loaded['program_number'] = loaded['program_number'].str.ljust(3)
    loaded = loaded.groupby(['race_date','race_number','program_number']).max()

    return loaded

In [4]:
## Ordering tracking table by date, number, program and trakus index
## Loading time: ~11 min
## Saved to csv 'ordered_tracking.csv' to save time 

# tracking_df = pd.read_csv(DATA_DIR / 'nyra_tracking_table.csv')
# ordered_tracking = tracking_df.groupby(['race_date','race_number','program_number','trakus_index']).min()
# ordered_tracking.to_csv(DATA_DIR / 'ordered_tracking.csv')


#### Other functions

In [5]:
# Let's def function to apply Haversine formula
# Passing a dataframe with coordinates (lat, lon)
# we get returned a list of distances between the coordinates

# this was modified from a snippet of code extracted from
# https://www.geeksforgeeks.org/program-distance-two-points-earth/ by Aarti_Rathi
def haversine_distance(coords):
    
    # Convert df from degrees to radians.
    rad_coords = np.radians(coords)

    # Create a shifted version of the coordinates to compare
    shifted_rad_coords = rad_coords.shift(1)

    # And subtract them to get our delta from point a to point b
    sub_coords = shifted_rad_coords.sub(rad_coords)

    dlat, dlon = sub_coords.latitude, sub_coords.longitude

    a = np.sin(dlat / 2)**2 + np.cos(rad_coords.latitude) * np.cos(shifted_rad_coords.latitude) * np.sin(dlon / 2)**2
    
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of earth in meters.
    r = 6371000
        
    # return the result list
    return (c * r).fillna('0').to_list()

## Creating new dataframes

In [6]:
# New dataframe filled only with info related to race and results.

df_results = load_data()

df_results.head(5)

track_id  distance_id course_type  \
race_date  race_number program_number                                     
2019-01-01 1           1                   AQU          650           D   
                       2                   AQU          650           D   
                       3                   AQU          650           D   
                       4                   AQU          650           D   
                       5                   AQU          650           D   

                                      track_condition  run_up_distance  \
race_date  race_number program_number                                    
2019-01-01 1           1                          MY                36   
                       2                          MY                36   
                       3                          MY                36   
                       4                          MY                36   
                       5                          MY                36   

                                      race_type  weight_carried  \
race_date  race_number program_number                             
2019-01-01 1           1                    AOC             123   
                       2                    AOC             120   
                       3                    AOC             118   
                       4                    AOC             123   
                       5                    AOC             118   

                                                jockey  odds  \
race_date  race_number program_number                          
2019-01-01 1           1                   Dylan Davis   130   
                       2               Junior Alvarado   295   
                       3                  Jose Lezcano   180   
                       4                  Jomar Garcia  1280   
                       5                 Manuel Franco  1150   

                                       position_at_finish  
race_date  race_number program_number                      
2019-01-01 1           1                                2  
                       2                                3  
                       3                                4  
                       4                                5  
                       5                                1

## Manipulating dataframes

In [9]:
%timeit haversine_distance(df_tracking.loc['2019-01-01',1,'3  '])

72.8 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
df_results.head(5)

track_id  distance_id course_type  \
race_date  race_number program_number                                     
2019-01-01 1           1                   AQU          650           D   
                       2                   AQU          650           D   
                       3                   AQU          650           D   
                       4                   AQU          650           D   
                       5                   AQU          650           D   

                                      track_condition  run_up_distance  \
race_date  race_number program_number                                    
2019-01-01 1           1                          MY                36   
                       2                          MY                36   
                       3                          MY                36   
                       4                          MY                36   
                       5                          MY                36   

                                      race_type  weight_carried  \
race_date  race_number program_number                             
2019-01-01 1           1                    AOC             123   
                       2                    AOC             120   
                       3                    AOC             118   
                       4                    AOC             123   
                       5                    AOC             118   

                                                jockey  odds  \
race_date  race_number program_number                          
2019-01-01 1           1                   Dylan Davis   130   
                       2               Junior Alvarado   295   
                       3                  Jose Lezcano   180   
                       4                  Jomar Garcia  1280   
                       5                 Manuel Franco  1150   

                                       position_at_finish  
race_date  race_number program_number                      
2019-01-01 1           1                                2  
                       2                                3  
                       3                                4  
                       4                                5  
                       5                                1

In [73]:
df_results.head(5)

track_id  distance_id course_type  \
race_date  race_number program_number                                     
2019-01-01 1           1                   AQU          650           D   
                       2                   AQU          650           D   
                       3                   AQU          650           D   
                       4                   AQU          650           D   
                       5                   AQU          650           D   

                                      track_condition  run_up_distance  \
race_date  race_number program_number                                    
2019-01-01 1           1                          MY                36   
                       2                          MY                36   
                       3                          MY                36   
                       4                          MY                36   
                       5                          MY                36   

                                      race_type  weight_carried  \
race_date  race_number program_number                             
2019-01-01 1           1                    AOC             123   
                       2                    AOC             120   
                       3                    AOC             118   
                       4                    AOC             123   
                       5                    AOC             118   

                                                jockey  odds  \
race_date  race_number program_number                          
2019-01-01 1           1                   Dylan Davis   130   
                       2               Junior Alvarado   295   
                       3                  Jose Lezcano   180   
                       4                  Jomar Garcia  1280   
                       5                 Manuel Franco  1150   

                                       position_at_finish  \
race_date  race_number program_number                       
2019-01-01 1           1                                2   
                       2                                3   
                       3                                4   
                       4                                5   
                       5                                1   

                                                                                   speed  \
race_date  race_number program_number                                                      
2019-01-01 1           1               [0.0, 2.116601375506467, 2.9395561051239127, 4...   
                       2                                                            None   
                       3                                                            None   
                       4                                                            None   
                       5                                                            None   

                                                                                distance  
race_date  race_number program_number                                                     
2019-01-01 1           1               [0, 0.5291503438766167, 0.7348890262809782, 1....  
                       2                                                            None  
                       3                                                            None  
                       4                                                            None  
                       5                                                            None

## Snippet testing

In [37]:
for i in df_results.head(500).index.to_list():
    idx = (i[0], i[1], str(i[2]).ljust(3))

    df_results['speed'].loc[idx] = haversine_distance(df_tracking.loc[idx])

In [71]:
def add_info():
    # Load tracking dataframe, may put it outside of function
    # df_tracking = pd.read_csv(DATA_DIR / 'ordered_tracking.csv')
    # df_tracking = df_tracking.groupby(['race_date','race_number','program_number','trakus_index']).min()

    # Add columns to df for new data
    if 'speed' not in df_results:
        df_results.insert(len(df_results.columns), column='speed', value=None)
    if 'distance' not in df_results:
        df_results.insert(len(df_results.columns), column='distance', value=None)

    # Loop target dataframe index as list
    # TODO modify it to work with a df passed to the function, maybe?
    for i in df_results.head(1).index.to_list():
        idx = (i[0], i[1], str(i[2]).ljust(3))
        distance = haversine_distance(df_tracking.loc[idx])

        df_results['distance'].loc[idx] = distance
        df_results['speed'].loc[idx] = np.array(distance, dtype='float64') * 4

In [ ]:
add_info()

In [18]:
df_results.loc['2019-01-01':'2019-01-31']

track_id  distance_id course_type  \
race_date  race_number program_number                                     
2019-01-01 1           3                   AQU          650           D   
                       4                   AQU          650           D   
                       5                   AQU          650           D   
                       1                   AQU          650           D   
                       2                   AQU          650           D   
...                                        ...          ...         ...   
2019-01-27 9           2                   AQU          800           D   
                       3                   AQU          800           D   
                       5                   AQU          800           D   
                       7                   AQU          800           D   
                       9                   AQU          800           D   

                                      track_condition  run_up_distance  \
race_date  race_number program_number                                    
2019-01-01 1           3                          MY                36   
                       4                          MY                36   
                       5                          MY                36   
                       1                          MY                36   
                       2                          MY                36   
...                                               ...              ...   
2019-01-27 9           2                          FT                54   
                       3                          FT                54   
                       5                          FT                54   
                       7                          FT                54   
                       9                          FT                54   

                                      race_type  weight_carried  \
race_date  race_number program_number                             
2019-01-01 1           3                    AOC             118   
                       4                    AOC             123   
                       5                    AOC             118   
                       1                    AOC             123   
                       2                    AOC             120   
...                                         ...             ...   
2019-01-27 9           2                    MSW             122   
                       3                    MSW             122   
                       5                    MSW             122   
                       7                    MSW             115   
                       9                    MSW             115   

                                                   jockey  odds  \
race_date  race_number program_number                             
2019-01-01 1           3                     Jose Lezcano   180   
                       4                     Jomar Garcia  1280   
                       5                    Manuel Franco  1150   
                       1                      Dylan Davis   130   
                       2                  Junior Alvarado   295   
...                                                   ...   ...   
2019-01-27 9           2                      Eric Cancel  1990   
                       3                     Jose Lezcano   860   
                       5                  Junior Alvarado   240   
                       7               Benjamin Hernandez  9525   
                       9                 Joey R. Martinez   980   

                                       position_at_finish  
race_date  race_number program_number                      
2019-01-01 1           3                                4  
                       4                                5  
                       5                                1  
                       1                               

In [14]:
df_results.loc['2019-12-31', 8]

,track_id,distance_id,course_type,track_condition,run_up_distance,race_type,weight_carried,jockey,odds,position_at_finish
program_number,,,,,,,,,,
2,AQU,600,D,SY,45,MCL,112,Luis Cardenas,1850,6
5,AQU,600,D,SY,45,MCL,122,Dalton Brown,8100,3
7,AQU,600,D,SY,45,MCL,122,Gary Richards,6650,10
10,AQU,600,D,SY,45,MCL,117,Leonardo Corujo,4725,9
1,AQU,600,D,SY,45,MCL,124,Reylu Gutierrez,710,7
11,AQU,600,D,SY,45,MCL,122,Manuel Franco,90,2
4,AQU,600,D,SY,45,MCL,122,Benjamin Hernandez,370,5
6,AQU,600,D,SY,45,MCL,122,Kendrick Carmouche,630,4
8,AQU,600,D,SY,45,MCL,122,Eric Cancel,1080,1


In [15]:
df_results.jockey.describe()

count             15081
unique              178
top       Manuel Franco
freq               1289
Name: jockey, dtype: object

In [16]:
df_results.loc[df_results.position_at_finish == 3].jockey.describe()

count              2022
unique               86
top       Manuel Franco
freq                195
Name: jockey, dtype: object

In [17]:
df_results.loc[df_results.position_at_finish == 3].jockey.value_counts()

Manuel Franco       195
Dylan Davis         153
Jose Lezcano        129
Junior Alvarado     124
Irad Ortiz Jr.      121
                   ... 
Thomas Garner         1
Ross Geraghty         1
William McCarthy      1
Nazario Alvarado      1
Wilmer A. Garcia      1
Name: jockey, Length: 86, dtype: int64